In [1]:
import os, torch
from xml.etree import ElementTree as ET
from torch.utils.data import Dataset,DataLoader,random_split
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
FOLDER_DATASET = "/home/kk/Desktop/usama/datasets/VMMRdb"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
NUM_CLASSES = None
EPOCHS = 15

In [3]:
print(DEVICE)

cuda


# Implementations

In [4]:
def test_accuracy(model, test_loader):
  with torch.no_grad():
      total_correct = 0
      total_samples = 0
      for images, labels in test_loader:
          images, labels = images.to(DEVICE), labels.to(DEVICE)
          outputs = model(images)
          _, predicted = torch.max(outputs, 1)
          total_correct += (predicted == labels).sum().item()
          total_samples += labels.size(0)

      accuracy = total_correct / total_samples
      print(f'Test Accuracy On best Model: {accuracy:.4f}')

In [5]:

class CustomDataset(Dataset):
    def __init__(self,X,y,all_classes, root_dir, transform=None ) -> None:
        self.X = X
        self.y = y
        self.root_dir = root_dir
        assert len(self.X) == len(self.y)

        self.transform = transform
        self.label_map = {label: idx for idx, label in enumerate(set(all_classes))}
        for i,label in enumerate(self.label_map.keys()):
          self.label_map[label] = i


    def __len__(self):
        return len(self.X)


    def __getitem__(self, idx):
        img_name = self.X[idx]
        label = self.y[idx]
        img_path = os.path.join(self.root_dir, label, img_name)

        label_arr = label.split('_')
        label = "_".join(label_arr[:-1])

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        else:
            image = transforms.ToTensor()(image)
        label = torch.tensor(self.label_map[label], dtype=torch.long)
        return image, label

In [6]:
import copy
from tqdm import tqdm

def train(epochs, model, criterion, optimizer, train_loader, valid_loader, test_loader):
  best_model = None
  best_acc = 0

  # Training loop
  for epoch in range(epochs):
      model.train()
      total_train_correct = 0
      total_train_samples = 0
      tqdm_train_loader = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}')
      for images, labels in tqdm_train_loader:
          images, labels = images.to(DEVICE), labels.to(DEVICE)
          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          _, predicted = torch.max(outputs, 1)
          total_train_correct += (predicted == labels).sum().item()
          total_train_samples += labels.size(0)
      train_accuracy = total_train_correct / total_train_samples

      # Validation loop
      model.eval()
      with torch.no_grad():
          total_correct = 0
          total_samples = 0
          for images, labels in valid_loader:
              images, labels = images.to(DEVICE), labels.to(DEVICE)
              outputs = model(images)
              _, predicted = torch.max(outputs, 1)
              total_correct += (predicted == labels).sum().item()
              total_samples += labels.size(0)

          accuracy = total_correct / total_samples
          print(f'Epoch [{epoch+1}/{epochs}], Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {accuracy:.4f}')

          if (accuracy > best_acc):
            print("New Best Model with Accuracy: ", accuracy)
            best_acc = accuracy
            best_model = copy.deepcopy(model)
            torch.save(best_model,"scratch_training_E"+str(epoch)+"pt")

  print("Training finished.")

  # Testing the model
  model.eval()
  with torch.no_grad():
      total_correct = 0
      total_samples = 0
      for images, labels in test_loader:
          images, labels = images.to(DEVICE), labels.to(DEVICE)
          outputs = best_model(images)
          _, predicted = torch.max(outputs, 1)
          total_correct += (predicted == labels).sum().item()
          total_samples += labels.size(0)

      accuracy = total_correct / total_samples
      print(f'Test Accuracy On best Model: {accuracy:.4f}')
  return best_model

# Main

In [7]:
classes_list = os.listdir(FOLDER_DATASET)
dataset_list = []

for _class in classes_list:
  images = os.listdir(os.path.join(FOLDER_DATASET,_class))
  for image in images:
    class_l =  _class.split("_")
    dataset_list.append((image,_class,"_".join(class_l[:-1])))
dataset_list = np.array(dataset_list)
NUM_CLASSES = len(np.unique(dataset_list[:,2]))

print("Dataset Images:",len(dataset_list), "TOTAL CLASSES: ", NUM_CLASSES)

Dataset Images: 285086 TOTAL CLASSES:  1175


In [8]:
X_train, X_test, y_train, y_test = train_test_split(dataset_list[:,0],dataset_list[:,1], test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = CustomDataset(X_train,y_train,dataset_list[:,2],FOLDER_DATASET, transform)
valid_dataset = CustomDataset(X_val, y_val,dataset_list[:,2], FOLDER_DATASET, transform)
test_dataset = CustomDataset(X_test,y_test,dataset_list[:,2], FOLDER_DATASET, transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Main

In [10]:
model = models.densenet121(weights=True)
for param in model.parameters():
    param.requires_grad = False
model.classifier = nn.Linear(model.classifier.in_features, NUM_CLASSES)
model = model.to(DEVICE)

/home/kk/miniconda3/envs/env_usama/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
best_model = train(EPOCHS,model,criterion,optimizer,train_loader,valid_loader,test_loader)

Epoch 1/15: 100%|██████████| 1337/1337 [1:10:54<00:00,  3.18s/it]


Epoch [1/15], Training Accuracy: 0.2074, Validation Accuracy: 0.2865
New Best Model with Accuracy:  0.2865110405668485


Epoch 2/15: 100%|██████████| 1337/1337 [1:04:00<00:00,  2.87s/it]


Epoch [2/15], Training Accuracy: 0.3395, Validation Accuracy: 0.3424
New Best Model with Accuracy:  0.3423715733903923


Epoch 3/15: 100%|██████████| 1337/1337 [1:01:38<00:00,  2.77s/it]


...

Epoch [10/15], Training Accuracy: 0.5063, Validation Accuracy: 0.3892


Epoch 11/15: 100%|██████████| 1337/1337 [1:01:34<00:00,  2.76s/it]


Epoch [11/15], Training Accuracy: 0.5116, Validation Accuracy: 0.3890


Epoch 12/15:  62%|██████▏   | 835/1337 [38:18<23:01,  2.75s/it]


KeyboardInterrupt: 

In [ ]:
torch.save(best_model,"vmmrdb_finetuning.pt")